In [0]:
# Preciso agora fazer a limpeza dos notebooks , devo fazer essa limpeza com as tabelas bronze transformando as em ouro no futuro
# dbutils.fs.mkdirs("dbfs:/mnt/datalake/bronze/postgres/vendas/clientes")
# dbutils.fs.mkdirs("dbfs:/mnt/datalake/bronze/postgres/vendas/produtos")
# dbutils.fs.mkdirs("dbfs:/mnt/datalake/bronze/postgres/vendas/pedidos")
# dbutils.fs.mkdirs("dbfs:/mnt/datalake/bronze/postgres/vendas/itens_pedido")



# Catálogo Clientes

##  Transformações aplicadas no DataFrame `df_clientes`
- **Coluna `nome`**
  - Remove caracteres invisíveis (`\u00A0`, `\u2000-\u200B`).
  - Aplica `trim` para retirar espaços extras.
  - Converte para formato capitalizado (`initcap`).

- **Coluna `cidade`**
  - Remove caracteres invisíveis.
  - Elimina prefixos como `da`, `de`, `do`, `das` no início.
  - Aplica `trim` e `initcap` para padronizar.

- **Coluna `email`**
  - Remove espaços extras (`trim`).
  - Converte para maiúsculas (`upper`).

- **Tratamento de valores nulos**
  - Substitui `NULL` em:
    - `email` → `"Desconhecido"`
    - `nome` → `"Desconhecido"`
    - `cidade` → `"Não Informado"`
  - Remove linhas sem `id` (`na.drop(subset=["id"])`).




In [0]:
df = spark.read.format("delta").load("dbfs:/mnt/datalake/bronze/postgres/vendas/clientes")
display(df)

# df.filter(col("nome").isNull()).show();

# #Filtra registros sem valores nulos 
# df.filter(col("nome").isNotNull()).show();

# #Conta quantos valores nulos existem em uma coluna 
# df.filter(col("nome").isNull()).count();

# # Filtra valrias colunas nulas de uma vez 
# df.filter(col("nome").isNull() | col("cidade").isNull()).show();




In [0]:
from pyspark.sql.functions import col, trim, initcap, regexp_replace, upper

df_clientes = (
    df.withColumn(
        "nome",
        initcap(
            trim(
                regexp_replace(col("nome"), r"[\u00A0\u2000-\u200B]", " ")
            )
        )
    )
    .withColumn(
        "cidade",
        initcap(
            trim(
                regexp_replace(
                    regexp_replace(col("cidade"), r"[\u00A0\u2000-\u200B]", " "), 
                    r"^(da|de|do|das)\s+", ""
                )
            )
        )
    )
    .withColumn("email", upper(trim(col("email"))))
    .fillna({"email": "Desconhecido", "nome": "Desconhecido", "cidade": "Não Informado"})
    .na.drop(subset=["id"])
)

display(df_clientes)

# silver_path = "dbfs:/mnt/datalake/silver/vendas/clientes"

# df_clientes.write \
#     .format("delta") \
#     .mode("overwrite") \
#     .option('mergeSchema', 'true').save(silver_path)

In [0]:
# %sql
# -- CREATE DATABASE IF NOT EXISTS silver;

# -- CREATE TABLE IF NOT EXISTS silver.vendas_clientes (
# --   id INT,
# --   nome STRING,
# --   email STRING,
# --   cidade STRING
# -- )




In [0]:
df_clientes.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("silver.vendas_clientes")

# Catálogo Produtos


In [0]:

df1 = spark.read.format("delta").load("dbfs:/mnt/datalake/bronze/postgres/vendas/produtos")
display(df1)

In [0]:
from pyspark.sql.functions import col, trim, initcap, regexp_replace, upper, when

df_produtos = (
    df1.withColumn(
        "nome",
        initcap(
            trim(
                regexp_replace(col("nome"), r"[\u00A0\u2000-\u200B]", " ")
            )
        )
    )
)


display(df_produtos)

